In [49]:
import math
import pandas as pd
from scipy.stats import norm
from statsmodels.stats.weightstats import _zconfint_generic, _tconfint_generic

In [2]:
data = []
with open('english-water.txt') as fp:
    print(fp.readline())
    for line in fp:
        parts = line.strip().split('\t')
        if not parts:
            continue

        data.append(dict(
            location=parts[0],
            town=parts[1],
            mortality=int(parts[2]),
            hardness=int(parts[3]),
        ))

data = pd.DataFrame(data)

location	town	mortality	hardness



In [3]:
data.head()

,location,town,mortality,hardness
0,South,Bath,1247,105
1,North,Birkenhead,1668,17
2,South,Birmingham,1466,5
3,North,Blackburn,1800,14
4,North,Blackpool,1609,18


In [4]:
data.describe()

,mortality,hardness
count,61.000000,61.000000
mean,1524.147541,47.180328
std,187.668754,38.093966
min,1096.000000,5.000000
25%,1379.000000,14.000000
50%,1555.000000,39.000000
75%,1668.000000,75.000000
max,1987.000000,138.000000


In [20]:
norm_low = norm.ppf(0.025)
norm_hi = norm.ppf(0.975)

In [25]:
norm_low

-1.9599639845400545

In [26]:
norm_hi

1.959963984540054

In [50]:
_tconfint_generic?

Signature: _tconfint_generic(mean, std_mean, dof, alpha, alternative)
Docstring:
generic t-confint based on summary statistic

Parameters
----------
mean : float or ndarray
    Value, for example mean, of the first sample.
std_mean : float or ndarray
    Standard error of the difference value1 - value2
dof : int or float
    Degrees of freedom
alpha : float
    Significance level for the confidence interval, coverage is
    ``1-alpha``.
alternative : str
    The alternative hypothesis, H1, has to be one of the following

       * 'two-sided' : H1: ``value1 - value2 - diff`` not equal to 0.
       * 'larger' :   H1: ``value1 - value2 - diff > 0``
       * 'smaller' :  H1: ``value1 - value2 - diff < 0``

Returns
-------
lower : float or ndarray
    Lower confidence limit. This is -inf for the one-sided alternative
    "smaller".
upper : float or ndarray
    Upper confidence limit. This is inf for the one-sided alternative
    "larger".
File:      ~/anaconda3/lib/python3.8/site-packages/s

In [21]:
mort_mean = data.mortality.mean()
mort_std = data.mortality.std()
mort_cnt = data.mortality.count()

In [51]:
_tconfint_generic(mort_mean, mort_std / math.sqrt(mort_cnt), mort_cnt - 1, 0.05, 'two-sided')

(1476.0833413552848, 1572.2117406119285)

In [28]:
mort_south_mean = data[data.location == 'South'].mortality.mean()
mort_south_std = data[data.location == 'South'].mortality.std()
mort_south_cnt = data[data.location == 'South'].mortality.count()

In [29]:
mort_south_cnt

26

In [52]:
_tconfint_generic(mort_south_mean, mort_south_std / math.sqrt(mort_south_cnt), mort_south_cnt - 1, 0.05, 'two-sided')

(1320.1517462936238, 1433.463638321761)

In [33]:
mort_north_mean = data[data.location == 'North'].mortality.mean()
mort_north_std = data[data.location == 'North'].mortality.std()
mort_north_cnt = data[data.location == 'North'].mortality.count()

In [34]:
mort_north_cnt

35

In [35]:
round(mort_north_mean + norm_low * mort_north_std / math.sqrt(mort_north_cnt), 4)

1588.2336

In [36]:
round(mort_north_mean + norm_hi * mort_north_std / math.sqrt(mort_north_cnt), 4)

1678.9664

In [37]:
hardness_stats = data.groupby('location')['hardness'].agg(['mean', 'std', 'count'])

In [38]:
hardness_stats

,mean,std,count
location,,,
North,30.400000,26.134494,35
South,69.769231,40.360682,26


In [46]:
for loc, row in hardness_stats.iterrows():
    print('Loc:', loc)
    print('Low:', row['mean'] + norm_low * row['std'] / math.sqrt(row['count']))
    print('Hi:', row['mean'] + norm_hi * row['std'] / math.sqrt(row['count']))

Loc: North
Low: 21.741789085882075
Hi: 39.058210914117915
Loc: South
Low: 54.25536932516893
Hi: 85.2830922132926
